In [1]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F
from torch_geometric.utils.convert import from_scipy_sparse_matrix, from_networkx
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
import torch.nn as nn


import random
import numpy as np
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip
from scipy import sparse
from torch.utils.data import random_split
import time

# load pickle module
import pickle
import networkx as nx
# from tqdm import tqdm
import sys
import h5py

In [10]:
%run ../../codes/GCN.py

In [7]:
lr_list = [np.power(0.5, i) for i in range(2, 16, 2)]*10

In [8]:
lr_list[3]

0.00390625

In [11]:
datafolder = '/Users/qingyao/../../Volumes/Seagate_Qing/branching_data/gnn_regression_2d5/including_large_r/'
    # datafolder = '/Users/qingyao/Documents/branching_data/gnn_regression/'

num_x = 50
layer = 4
# save_dir = '/rds/general/user/qy1815/home/branching_superspreading/regression_ng_{}_layer_{}/'.format(
#num_x, layer)

job_idx = 100 #int(sys.argv[1])-1  # ensemble/parameter index
s = job_idx//70
es_idx = 4 # job_idx % 70  # ensemble/parameter index
s = 1 #int(s)  # seed index

    # load data
    # WN = np.loadtxt('W_avg.csv')
dataset = torch.load(datafolder+'dataset_{}.pt'.format(num_x))
lr_list = [np.power(0.5, i) for i in range(2, 16, 2)]*10
my_lr =lr_list[3] #lr_list[es_idx]

torch.manual_seed(s)
all_data_len = len(dataset)
train_size = int(all_data_len * 0.6)
val_size = int(all_data_len * 0.2)
test_size = all_data_len - train_size - val_size

train_data, val_data, test_data = random_split(
        dataset, [train_size, val_size, test_size])

    # Now we can create a DataLoader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=True)
    # Create a model and an optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # model = GCN_ng(num_node_features=num_x, hidden_channels=[
    #                128, 64, 16, 8], num_hlayers=layer).to(device)
# model = GCN_ng(num_node_features=num_x, hidden_channels=[
#         128, 64, 16, 8], num_hlayers=layer).to(device)
model = MLP(num_node_features=num_x, hidden_channels=[
        128, 64, 16, 8], num_hlayers=layer).to(device)
optimizer = torch.optim.Adam(
        model.parameters(), lr=my_lr, weight_decay=5e-4)

    # training and validation
counter = 0
count_epochs = 0
best = float("inf")
epochs = 100
patience = 10
loss_ep = []

seed = int(time.time()) + es_idx
# random.seed(seed)
torch.manual_seed(seed)

for epoch in range(epochs):
    loss, myres, reals = train(model, train_loader, optimizer, device)
    # loss_ep.append(loss)
    val_loss = validate(model, val_loader, device)
    loss_ep.append(val_loss)
    print(epoch, val_loss)
    if val_loss < best:
        best = val_loss
        counter = 0
        # Save the best model
#         torch.save(model.state_dict(
#         ), save_dir+'best_model_{}_{}.pth'.format(s, es_idx))
    else:
        counter += 1
        count_epochs += 1

    if counter > patience:
        break

0 3.9779885095263285
1 3.491033883321853
2 3.1248532030317517
3 3.474594283482385
4 1.0255534266668653
5 0.8907242190270197
6 1.1505173448532346
7 0.8819281697273255
8 0.5082200515837897
9 0.536661584415133
10 0.48977221269456167
11 0.5925142626913767
12 0.47678991442634944
13 0.7193532680708264
14 0.507485590473054
15 0.545212271478441
16 0.9486823097107903
17 0.4254423114042433
18 0.493947234229436
19 0.9107219423566546
20 0.4273051296907758
21 0.39634876336370195
22 0.49699782227712963
23 0.3987997024778336
24 0.4114665447719513
25 0.5734893776121593
26 0.5243418422956315
27 0.4245579908764552
28 0.41444656101484145
29 0.4155509608132499
30 0.4224038805280413
31 0.3771170937825763
32 0.4029270559076279
33 0.3679647201583499
34 0.4079960190114521
35 0.4726111044959416
36 0.3599187557659452
37 0.34537507882193913
38 0.40487726245607647
39 0.34023501750022644
40 0.40216155487393573
41 0.35426377143178667
42 0.35185880878615
43 0.34592164981932866
44 0.519668349765596
45 0.3494712692404